# Table of Contents
* [1.Library Import](#chapter1)
* [2.Function Definition](#chapter2)
* [3.Searching and Downloading L2 Products](#chapter3)
* [4.Searching and Downloading L3 Products](#chapter4)
   

In [3]:
%%html
<style>
body {font-family: Trebuchet MS}
p {font-family: Trebuchet MS}
h1   {color:black;background-color:}
.exo {background-color:red;border: none;color: white;padding: 15px 32px;
  text-align: center;text-decoration: none;display: inline-block;font-size: 16px;
  margin: 4px 2px;}
  
.trick {background-color:yellow;border: none;color: red;padding: 10px 10px;
  text-align: center;text-decoration: none;display: inline-block;font-size: 20px;
  margin: 4px 2px;}

</style>

### © All the code and part of the notebook was created by Nusrat Jahan, PhD student at LIENSs, La Rochelle. 


This notebook contains some examples how to search and download SWOT data from Aviso website. In order to access Aviso database, one must have an Aviso+ account. The user can provide user id and password directly or can keep them on the swot.ini configuration file [ftp_aviso_plus].

The current status of publicly available data in Aviso database are given below:

| Product level |  Version   |       Orbit type       |  Product type  |
|:-------------:|:----------:|:----------------------:|:--------------:|
|      L2       |    PIB0    | Science (21 day orbit) |     basic      |
|               |            |  calval (1 day orbit)  |     expert     |
|               |            |                        |   unsmoothed   |
|               |            |                        |   wind-wave    |
|      L3       | alpha_v0_3 | Science (21 day orbit) |     basic      |
|               |            |  calval (1 day orbit)  |     expert     |



# 1. Library import

In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
from ftplib import FTP
import sys
sys.path.append('C:/Academic La Rochelle/M2/Internship/Data/swot/')

# 2. Function Definition 

1. **`connect_ftp` Function:**
     - Establishes a connection to an FTP (File Transfer Protocol) server.
     - Checks if `userid` and `userpass` are provided (and are strings or lists).
     - Attempts to connect to the FTP server at `ftp_host`.
     - Tries to log in with the provided `userid` and `userpass`.
     - Returns the `FTP` object if successful, otherwise prints an error message.
     

2. **`get_data_dir` Function:**
   - Constructs a directory path based on specified parameters, used to navigate within the FTP server.
   - Constructs and returns a string representing a directory path based on the provided parameters.
   - If the parameters don't match expected values, it prints an error message.
   

3. **`ftp_data_query` Function:**
   - Queries the FTP server for files in a specific directory and extracts relevant information.
   - Lists all files in the specified `data_dir`.
   - Extracts and processes information from file names, such as cycle numbers, pass IDs, start and end times.
   - Returns a pandas DataFrame containing this extracted information.
   

4. **`filter_cycle` Function:**
   - Filters the provided DataFrame based on specified cycle numbers.
   - If `cycle` is 'all', sorts `df` by `sorting_col` and resets the index as specified.
   - If `cycle` is a list of specific cycles, filters `df` to include only those cycles, sorts it, and resets the index.
   - Returns the filtered and sorted DataFrame.



In [2]:
def connect_ftp(userid=None, userpass=None, ftp_host='ftp-access.aviso.altimetry.fr'):

    if isinstance(userid, (str, list)) & isinstance(userpass, (str, list)):
        ftp = FTP(ftp_host)
        ftp.login(user=userid, passwd=userpass)
    else:
        print('Provide valid userid and password')
    return ftp


def get_data_dir(plevel, orbit, ptype, version):
    if 'l2' in plevel:
        if orbit == 'science':
            return f'/swot_beta_products/l2_karin/l2_lr_ssh/{version}/21day_orbit/{ptype}'
        elif orbit == 'calval':
            return f'/swot_beta_products/l2_karin/l2_lr_ssh/{version}/1day_orbit/{ptype}'
        else:
            print("Not a valid orbit type, provide either 'science' or 'calval'.")
    if 'l3' in plevel:
        if orbit == 'science':
            return f'/swot_beta_products/l3_karin_nadir/21day_orbit/{ptype}/{version}'
        elif orbit == 'calval':
            return f'/swot_beta_products/l3_karin_nadir/1day_orbit/{ptype}/{version}'
        else:
            print("Not a valid orbit type, provide either 'science' or 'calval'.")


def ftp_data_query(ftp, data_dir):
    files = ftp.nlst(data_dir)
    fnames = np.array([file.split('/')[-1] for file in files])
    cycles = np.array([int(file.split('/')[-1].split('_')[5]) for file in files])
    passIDs = np.array([int(file.split('/')[-1].split('_')[6]) for file in files])
    start_time = np.array([pd.to_datetime(file.split('/')[-1].split('_')[7]) for file in files])
    end_time = np.array([pd.to_datetime(file.split('/')[-1].split('_')[8]) for file in files])

    return pd.DataFrame(dict(cycle=cycles, passID=passIDs, start_time=start_time, end_time=end_time, file_names=fnames))


def filter_cycle(df, cycle, reset_idx=['passID', 'cycle'], sorting_col=['passID']):
    if cycle == 'all':
        df = df.sort_values(by=sorting_col)
        df.set_index(reset_idx, inplace=True)
        return df
    else:
        df_c = pd.DataFrame()
        for c in cycle:
            temp_df = df[df.cycle == c]
            df_c = pd.concat([df_c, temp_df])
        df_c = df_c.sort_values(by=sorting_col)
        df_c.set_index(reset_idx, inplace=True)
        return df_c


5. **`download_files` Function:**
   - **Purpose:** Downloads files from an FTP server to a specified local directory.
   - Iterates through each file in the `files` list.
   - For each file, it opens (or creates) a file in the `destination_path` and writes the binary data retrieved from the FTP server.
   - Uses the FTP `RETR` command to retrieve each file.

6. **`search` Function:**
   - Searches for available data files on an FTP server based on specified criteria.
   - Connects to the FTP server using the provided credentials.
   - Constructs a data directory path based on the given parameters.
   - Queries the FTP server for files in the constructed directory.
   - Filters the query results based on the specified `passID` and `cycle`.
   - Closes the FTP connection.
   - Prints the filtered data if `print_result` is `True`.
   - Returns a DataFrame with the query results.

7. **`download` Function:**
   - Function for  download files from an FTP server.
   - Calls the `search` function to find files based on the specified criteria.
   - Establishes an FTP connection and changes the working directory to the relevant data directory.
   - Retrieves the list of filenames from the search results.
   - Downloads each file to the specified `destination_path` using the `download_files` function.
   - Closes the FTP connection.



In [3]:
def download_files(ftp, files, destination_path):
    for file in files:
        print("....downloading file ", file)
        with open(destination_path + f"/{file}", 'wb') as lf:
            ftp.retrbinary("RETR " + file, lf.write)
            



def search(orbit, plevel, ptype, version='alpha_v0_3', passID='all', cycle='all', ftp_userid=None, ftp_userpass=None,
           print_result=True):
    # user ID and password for Aviso plus. need to be in swot.ini file under [ftp_aviso_plus] if not given by the user directly.
    ftp = connect_ftp(ftp_userid, ftp_userpass)
    data_dir = get_data_dir(plevel, orbit, ptype, version)
    df_query = ftp_data_query(ftp, data_dir)
    ftp.close()

    if passID == 'all':
        df = filter_cycle(df_query, cycle, reset_idx=['cycle', 'passID'], sorting_col=['cycle'])
    else:
        df = pd.DataFrame()
        for id in passID:
            temp_df = df_query[df_query.passID == id]
            df = pd.concat([df, temp_df])

        df = filter_cycle(df, cycle, sorting_col=['cycle'])

    if len(df) != 0:
        if print_result:
            print(f'Available data on the server:')
            return df
        else:
            return df

    else:
        print(f'There is no data available for pass:{passID} and cycle:{cycle}.')


def download(destination_path, orbit, plevel, ptype, version='alpha_v0_3', passID='all', cycle='all',
             ftp_userid=None, ftp_userpass=None):

    df_query = search(orbit, plevel, ptype, version, passID, cycle, print_result=False)
    data_dir =get_data_dir(plevel, orbit, ptype, version)
    ftp = connect_ftp(ftp_userid, ftp_userpass)
    ftp.cwd(data_dir)

    files = df_query.file_names.values

    download_files(ftp, files, destination_path)

    ftp.close()

# 3. Searching and downloading L2 products

## Example: Searching and dwonloading available 'basic' product for specific pass/passes (here we are looking for pass 348 and 419) in science orbit.

__The options for different argument (i.e., orbit, product level, product type, product version etc.) can be obtained from the table above.__

The query dataset contains passId, cycle, starting & ending time of the passes and corresponding file names.

In [4]:
#Provide user id and password here 

userid = 'md.islam@etudiant.univ-lr.fr'
userpass = '3A8JFo'
search(orbit='science',plevel='l2',ptype='basic', version='PIB0', passID=[564], ftp_userid=userid, ftp_userpass=userpass)

Available data on the server:


start_time            end_time  \
passID cycle                                           
564    3     2023-09-21 01:48:39 2023-09-21 02:39:25   
       4     2023-10-11 22:33:41 2023-10-11 23:24:26   
       5     2023-11-01 19:18:46 2023-11-01 20:09:32   

                                                     file_names  
passID cycle                                                     
564    3      SWOT_L2_LR_SSH_Basic_003_564_20230921T014839_2...  
       4      SWOT_L2_LR_SSH_Basic_004_564_20231011T223341_2...  
       5      SWOT_L2_LR_SSH_Basic_005_564_20231101T191846_2...

So, for pass 348, SWOT products from cycle 4 to 6 and for pass 419 only cycle 3 are available. Now one can download any available data as following 

In [ ]:
# one must define a destination path
destination_path = r"T:\swot\data\SWOT_L2_LR"
# using the download function to download all available data from the query
download(destination_path, orbit='science',plevel='l2',ptype='basic', version='PIB0', passID=[348, 419])

In [ ]:
# To download any specific cycle for a specific file. For example, for pass 348, cycle 4
download(destination_path, orbit='science',plevel='l2',ptype='basic', version='PIB0', passID=[348], cycle=[4])

- One can change the ptype to 'expert' or 'unsmoothed' or 'wind-wave' to search and download any of these types of products. 
- Similarly, one can change the orbit to 'calval' to access the data from Cal/Val phase. 
- As for version, right now 'PIB0' is the only publicly available data on the Aviso database.

# 4. Searching and downloading L3 products
## Example: Searching and dwonloading available 'basic' product for specific pass/passes (here we are looking for pass 189) in science orbit.

In [5]:
#Provide user id and password
#userid = None
#userpass = None
search(orbit='science',plevel='l3',ptype='basic', version='alpha_v0_3', passID=[564],cycle='all', ftp_userid=userid, ftp_userpass=userpass)

error_temp: 450 /swot_beta_products/l3_karin_nadir/21day_orbit/basic/alpha_v0_3: No such file or directory

So, for pass 189, SWOT products from cycle 3 to 6  are available. Now one can download any available data as following 

In [ ]:
# one must define a destination path
destination_path = r"C:/Academic La Rochelle/M2/Internship/Data/swot/"
# to download one or more specific cycle
download(destination_path, orbit='science',plevel='l3',ptype='basic', version='alpha_v0_3', passID=[189], cycle='all', ftp_userid=userid, ftp_userpass=userpass)